In [1]:
import json

from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp

import matplotlib.pyplot as plt

from pepbench.io import load_challenge_results_from_folder
from pepbench.io import convert_hz_to_ms
from pepbench.datasets import EmpkinsDataset, GuardianDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
root_path = Path("../../")

In [3]:
result_path = root_path.joinpath("results")
result_path

WindowsPath('../../results')

In [4]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(empkins_base_path)

\Users\sebas\Development\ResearchInternship\Data\2024_08_PEP_Benchmarking\EmpkinS_Dataset


In [5]:
save_results = True

In [6]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
fs_empkins = dataset_empkins.sampling_rate_icg
print(f"Sampling rate ICG: {fs_empkins}")
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
fs_guardian = dataset_guardian.sampling_rate_icg
print(f"Sampling rate ICG: {fs_guardian}")

Sampling rate ICG: 1000
Sampling rate ICG: 500


In [7]:
algo_levels = ["q_wave_algorithm", "b_point_algorithm", "outlier_correction_algorithm"]
algo_level_mapping = dict(zip(algo_levels, ["Q-Wave Algorithm", "B-Point Algorithm", "Outlier Correction"]))

In [8]:
results_empkins = load_challenge_results_from_folder(
    result_path.joinpath("empkins_dataset_reference_q_wave").resolve(), index_cols_per_sample=["participant", "condition", "phase"]
).per_sample
results_guardian = load_challenge_results_from_folder(
    result_path.joinpath("guardian_dataset_reference_q_wave").resolve(), index_cols_per_sample=["participant", "phase"]
).per_sample

In [9]:
results_empkins = results_empkins.droplevel("condition")

In [10]:
results_empkins_norm = results_empkins.copy()
results_guardian_norm = results_guardian.copy()
results_empkins_norm[("b_point_sample", "estimated")] = results_empkins[("b_point_sample", "estimated")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_empkins_norm[("b_point_sample", "reference")] = results_empkins[("b_point_sample", "reference")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("b_point_sample", "estimated")] = results_guardian[("b_point_sample", "estimated")] - results_guardian[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("b_point_sample", "reference")] = results_guardian[("b_point_sample", "reference")] - results_guardian[("heartbeat_start_sample", "estimated")]

In [11]:
results_guardian_norm

heartbeat_id  \
                                                                                                         estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                      
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Pause    0             0   
                                                                                                   1             1   
                                                                                                   2             2   
                                                                                                   3             3   
                                                                                                   4             4   
...                                                                                                            ...   
                 stern1985                       none                         GDN0030     TiltDown 51           51   
                                                                                                   52           52   
                                                                                                   53           53   
                                                                                                   54           54   
                                                                                                   55           55   

                                                                                                                 \
                                                                                                      reference   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                   
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Pause    0          0   
                                                                                                   1          1   
                                                                                                   2          2   
                                                                                                   3          3   
                                                                                                   4          4   
...                                                                                                         ...   
                 stern1985                       none                         GDN0030     TiltDown 51        51   
                                                                                                   52        52   
                                                                                                   53        53   
                                                                                                   54        54   
                                                                                                   55        55   

                                                                                                      heartbeat_start_sample  \
                                                                                                                   estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                                
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Pause    0                     352   
                                                                                                   1                     808   
                                                                                                   2                    1252   
                                                                      

In [12]:
#results_empkins_cleaned = results_empkins_norm.dropna(subset=[("b_point_sample", "estimated")])
#results_guardian_cleaned = results_guardian_norm.dropna(subset=[("b_point_sample", "estimated")])
results_empkins_cleaned = results_empkins_norm
results_guardian_cleaned = results_guardian_norm

In [13]:
cols = [("heartbeat_start_sample", "estimated"), ("heartbeat_start_sample", "reference"), ("heartbeat_end_sample", "estimated"), ("heartbeat_end_sample", "reference"), ("q_wave_onset_sample", "estimated"), ("q_wave_onset_sample", "reference"), ("b_point_sample", "estimated"), ("b_point_sample", "reference"), ("pep_sample", "estimated"), ("pep_sample", "reference")]
results_empkins_cleaned_ms = results_empkins_cleaned.copy()
results_guardian_cleaned_ms = results_guardian_cleaned.copy()
results_empkins_cleaned_ms[cols] = results_empkins_cleaned[cols] * convert_hz_to_ms(sampling_frequency=fs_empkins)
results_guardian_cleaned_ms[cols] = results_guardian_cleaned[cols] * convert_hz_to_ms(sampling_frequency=fs_guardian)

In [14]:
results_guardian_cleaned_ms

heartbeat_id  \
                                                                                                         estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                      
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Pause    0             0   
                                                                                                   1             1   
                                                                                                   2             2   
                                                                                                   3             3   
                                                                                                   4             4   
...                                                                                                            ...   
                 stern1985                       none                         GDN0030     TiltDown 51           51   
                                                                                                   52           52   
                                                                                                   53           53   
                                                                                                   54           54   
                                                                                                   55           55   

                                                                                                                 \
                                                                                                      reference   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                   
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Pause    0          0   
                                                                                                   1          1   
                                                                                                   2          2   
                                                                                                   3          3   
                                                                                                   4          4   
...                                                                                                         ...   
                 stern1985                       none                         GDN0030     TiltDown 51        51   
                                                                                                   52        52   
                                                                                                   53        53   
                                                                                                   54        54   
                                                                                                   55        55   

                                                                                                      heartbeat_start_sample  \
                                                                                                                   estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                                
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Pause    0                   704.0   
                                                                                                   1                  1616.0   
                                                                                                   2                  2504.0   
                                                                      

In [15]:
preprocessed_results_empkins_guardian = pd.concat([results_empkins_cleaned_ms, results_guardian_cleaned_ms])
preprocessed_results_empkins_guardian

heartbeat_id  \
                                                                                                         estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                      
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      Prep     0             0   
                                                                                                   1             1   
                                                                                                   2             2   
                                                                                                   3             3   
                                                                                                   4             4   
...                                                                                                            ...   
                 stern1985                       none                         GDN0030     TiltDown 51           51   
                                                                                                   52           52   
                                                                                                   53           53   
                                                                                                   54           54   
                                                                                                   55           55   

                                                                                                                 \
                                                                                                      reference   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                   
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      Prep     0          0   
                                                                                                   1          1   
                                                                                                   2          2   
                                                                                                   3          3   
                                                                                                   4          4   
...                                                                                                         ...   
                 stern1985                       none                         GDN0030     TiltDown 51        51   
                                                                                                   52        52   
                                                                                                   53        53   
                                                                                                   54        54   
                                                                                                   55        55   

                                                                                                      heartbeat_start_sample  \
                                                                                                                   estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                                
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      Prep     0                   394.0   
                                                                                                   1                   984.0   
                                                                                                   2                  1569.0   
                                                                      

In [16]:
print(preprocessed_results_empkins_guardian.shape[0] - (results_empkins_cleaned_ms.shape[0] + results_guardian_cleaned_ms.shape[0]))

0


In [18]:
input_data = preprocessed_results_empkins_guardian

In [19]:
input_data = input_data[[("heartbeat_id", "reference"), ("b_point_sample", "estimated"), ("b_point_sample", "reference")]]
input_data.columns = input_data.columns.to_flat_index().str.join("_")
input_data.droplevel("q_wave_algorithm")
input_data

heartbeat_id_reference  \
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                                 
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      Prep     0                        0   
                                                                                                   1                        1   
                                                                                                   2                        2   
                                                                                                   3                        3   
                                                                                                   4                        4   
...                                                                                                                       ...   
                 stern1985                       none                         GDN0030     TiltDown 51                      51   
                                                                                                   52                      52   
                                                                                                   53                      53   
                                                                                                   54                      54   
                                                                                                   55                      55   

                                                                                                       b_point_sample_estimated  \
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                                   
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      Prep     0                      275.0   
                                                                                                   1                      254.0   
                                                                                                   2                      265.0   
                                                                                                   3                      260.0   
                                                                                                   4                      259.0   
...                                                                                                                         ...   
                 stern1985                       none                         GDN0030     TiltDown 51                     496.0   
                                                                                                   52                     436.0   
                                                                                                   53                     454.0   
                                                                                                   54                     388.0   
                                                                                                   55                     394.0   

                                                                                                       b_point_sample_reference  
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                                  
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      Prep     0                      240.0  
                                                                                                   1                      222.0  
                                                                                                   2                      242.0  
                                                                       

In [20]:
input_data["algo_combis"] = input_data.index.get_level_values("b_point_algorithm") + "_" + input_data.index.get_level_values("outlier_correction_algorithm")
input_data = input_data.reset_index().drop(columns=["b_point_algorithm", "outlier_correction_algorithm"])
input_data

C:\Users\sebas\AppData\Local\Temp\ipykernel_47796\3413036251.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["algo_combis"] = input_data.index.get_level_values("b_point_algorithm") + "_" + input_data.index.get_level_values("outlier_correction_algorithm")


q_wave_algorithm participant     phase  level_5  \
0       q-wave-reference      VP_001      Prep        0   
1       q-wave-reference      VP_001      Prep        1   
2       q-wave-reference      VP_001      Prep        2   
3       q-wave-reference      VP_001      Prep        3   
4       q-wave-reference      VP_001      Prep        4   
...                  ...         ...       ...      ...   
348325  q-wave-reference     GDN0030  TiltDown       51   
348326  q-wave-reference     GDN0030  TiltDown       52   
348327  q-wave-reference     GDN0030  TiltDown       53   
348328  q-wave-reference     GDN0030  TiltDown       54   
348329  q-wave-reference     GDN0030  TiltDown       55   

        heartbeat_id_reference  b_point_sample_estimated  \
0                            0                     275.0   
1                            1                     254.0   
2                            2                     265.0   
3                            3                     260.0   
4                            4                     259.0   
...                        ...                       ...   
348325                      51                     496.0   
348326                      52                     436.0   
348327                      53                     454.0   
348328                      54                     388.0   
348329                      55                     394.0   

        b_point_sample_reference  \
0                          240.0   
1                          222.0   
2                          242.0   
3                          248.0   
4                          242.0   
...                          ...   
348325                     498.0   
348326                     490.0   
348327                     454.0   
348328                     430.0   
348329                     444.0   

                                            algo_combis  
0       arbol2017-isoelectric-crossings_forouzanfar2018  
1       arbol2017-isoelectric-crossings_forouzanfar2018  
2       arbol2017-isoelectric-crossings_forouzanfar2018  
3       arbol2017-isoelectric-crossings_forouzanfar2018  
4       arbol2017-isoelectric-crossings_forouzanfar2018  
...                                                 ...  
348325                                   stern1985_none  
348326                                   stern1985_none  
348327                                   stern1985_none  
348328                                   stern1985_none  
348329                                   stern1985_none  

[348330 rows x 8 columns]

In [21]:
# Pivot the DataFrame
input_data_wide = input_data.pivot_table(
    index=['participant', 'phase', 'heartbeat_id_reference', 'b_point_sample_reference'],
    columns="algo_combis",
    values='b_point_sample_estimated'
).reset_index()

# Flatten the columns
input_data_wide.columns.name = None
input_data_wide.columns = [f'{col}' if isinstance(col, str) else f'{col[1]}' for col in input_data_wide.columns]
input_data_wide

participant          phase  heartbeat_id_reference  \
0         GDN0005  HoldingBreath                       0   
1         GDN0005  HoldingBreath                       1   
2         GDN0005  HoldingBreath                       3   
3         GDN0005  HoldingBreath                       4   
4         GDN0005  HoldingBreath                       5   
...           ...            ...                     ...   
11219      VP_032           Talk                      39   
11220      VP_032           Talk                      40   
11221      VP_032           Talk                      41   
11222      VP_032           Talk                      42   
11223      VP_032           Talk                      43   

       b_point_sample_reference  \
0                         388.0   
1                         404.0   
2                         376.0   
3                         390.0   
4                         386.0   
...                         ...   
11219                     310.0   
11220                     322.0   
11221                     340.0   
11222                     311.0   
11223                     307.0   

       arbol2017-isoelectric-crossings_forouzanfar2018  \
0                                                438.0   
1                                                420.0   
2                                                382.0   
3                                                394.0   
4                                                398.0   
...                                                ...   
11219                                            335.0   
11220                                            331.0   
11221                                            317.0   
11222                                            365.0   
11223                                            342.0   

       arbol2017-isoelectric-crossings_linear-interpolation  \
0                                                  438.0      
1                                                  422.0      
2                                                  382.0      
3                                                  394.0      
4                                                  398.0      
...                                                  ...      
11219                                              335.0      
11220                                              331.0      
11221                                              317.0      
11222                                              365.0      
11223                                              342.0      

       arbol2017-isoelectric-crossings_none  \
0                                     438.0   
1                                     340.0   
2                                     382.0   
3                                     394.0   
4                                     398.0   
...                                     ...   
11219                                 335.0   
11220                                 331.0   
11221                                 317.0   
11222                                 365.0   
11223                                 342.0   

       arbol2017-second-derivative_forouzanfar2018  \
0                                            398.0   
1                                            350.0   
2                                            296.0   
3                                            344.0   
4                                            312.0   
...                                            ...   
11219                                        276.0   
11220                                        298.0   
11221                                        300.0   
11222                                        324.0   
11223                                        313.0   

       arbol2017-second-derivative_linear-interpolation  \
0                                                 398.0   
1                                                 350.0   
2                  

In [22]:
input_data_wide = input_data_wide.dropna()
input_data_wide

participant          phase  heartbeat_id_reference  \
0         GDN0005  HoldingBreath                       0   
1         GDN0005  HoldingBreath                       1   
2         GDN0005  HoldingBreath                       3   
3         GDN0005  HoldingBreath                       4   
4         GDN0005  HoldingBreath                       5   
...           ...            ...                     ...   
11219      VP_032           Talk                      39   
11220      VP_032           Talk                      40   
11221      VP_032           Talk                      41   
11222      VP_032           Talk                      42   
11223      VP_032           Talk                      43   

       b_point_sample_reference  \
0                         388.0   
1                         404.0   
2                         376.0   
3                         390.0   
4                         386.0   
...                         ...   
11219                     310.0   
11220                     322.0   
11221                     340.0   
11222                     311.0   
11223                     307.0   

       arbol2017-isoelectric-crossings_forouzanfar2018  \
0                                                438.0   
1                                                420.0   
2                                                382.0   
3                                                394.0   
4                                                398.0   
...                                                ...   
11219                                            335.0   
11220                                            331.0   
11221                                            317.0   
11222                                            365.0   
11223                                            342.0   

       arbol2017-isoelectric-crossings_linear-interpolation  \
0                                                  438.0      
1                                                  422.0      
2                                                  382.0      
3                                                  394.0      
4                                                  398.0      
...                                                  ...      
11219                                              335.0      
11220                                              331.0      
11221                                              317.0      
11222                                              365.0      
11223                                              342.0      

       arbol2017-isoelectric-crossings_none  \
0                                     438.0   
1                                     340.0   
2                                     382.0   
3                                     394.0   
4                                     398.0   
...                                     ...   
11219                                 335.0   
11220                                 331.0   
11221                                 317.0   
11222                                 365.0   
11223                                 342.0   

       arbol2017-second-derivative_forouzanfar2018  \
0                                            398.0   
1                                            350.0   
2                                            296.0   
3                                            344.0   
4                                            312.0   
...                                            ...   
11219                                        276.0   
11220                                        298.0   
11221                                        300.0   
11222                                        324.0   
11223                                        313.0   

       arbol2017-second-derivative_linear-interpolation  \
0                                                 398.0   
1                                                 350.0   
2                  

In [24]:
if save_results:
    input_data_wide.to_csv(result_path.joinpath("data/train_data.csv"))